## Python Strings but better -- mutable

#### Writing in pseudopython
- Want to be able to init str
- want to be able to change char at index
- want to be able to append

In [1]:
s = "abc"
s[1] = 'z' 
s.append('d')

TypeError: 'str' object does not support item assignment

## Using str.c to create string:

In [ ]:
%%file week6.c

#include "mystr.h"
#include <stdio.h>

int main() {
    mystr *p_s;
    create_string(&p_s);
    append_str(p_s, "hi");
    printf("The string is %s\n", get_str(p_s));
    printf("The 2nd character is %c\n", get_char(p_s, 1));
    set_char(p_s, 1, 'N');

    //An array of strings
    mystr arr[40]; //Cannot really do this because need to create the block as well
    
    mystr *str_block = (mystr**)malloc(40*sizeof(mystr*));
    //str_block is a block of ptr to mystr
    //Set str_block[0], str_block[1], ..., str_block[39] to be a valid pointer
    for (int i = 0; i < 40; i++) {
        create_string(&(str_block[i]));
        //Equivalently: create_string(str_block+i)
    }

    destroy_string(p_s);
}

In [3]:
%%bash
gcc week6.c -g -o week6
./week6

Undefined symbols for architecture arm64:
  "_main", referenced from:
      <initial-undefines>
ld: symbol(s) not found for architecture arm64
clang: error: linker command failed with exit code 1 (use -v to see invocation)
bash: line 2: ./week6: No such file or directory


CalledProcessError: Command 'b'gcc week6.c -g -o week6\n./week6\n'' returned non-zero exit status 127.

## ADTs

- An abstract data type (ADT) is any collection of values, tgt with operations on those values
    - ex: `int` with operations + - * / %
    - `lists` with operations `insert`, `remove`, `get`

- An ADT specifies what values are represented and what operations can be performed, but not how to store the values or how to carry out the operations

- ADTs are important for specification: they provide modularity and reuse -- usage is independent of implementation

- A data structure is an implementation of an ADT -- a way to represent the values, and algorithms for each operation

## Lists

- A list is an order collection of data items that supports the following operations:
    - `insert(list, i, x)`: Add item x at index i
    - `remove(list, i)`: Remove the item at index i
    - `get(list, i)`: return the item at index i 

- Defined ADT in a very abstract way, only specifiying what the outcome of each operation should be 
- Internally, this behaviour can be implemented any way we like -- in particular, lists can be implemented using arrays


Considerations
1) Speed/complexity
2) Memory usage/space
3) Ease of use


[10, 12, 45, 12, 20, 125]

Complexity of remove element at index i = O(n) 

Space requirement for block-of-memory implementation for list of size(n)
n*sizeof(int), so O(n)

Complexity of insert at index i:
O(n-i) //Need to move n-i arguments to the right
but really:
O(n) //Might need to move the entire block

Complexity of get at index i:
O(1)
- Need to compute the location of element at index i
- Achievable using pointer arithmetic: ptr + sizeof(int)*i


## Linked Lists

- Cannot add an element to an array/block of memory because there may not be space there.
- To remove an element from an array/block, need to shift the entire block to the left in memory
- Therefore, introduce **Linked Lists** to store the data instead
- Each item is stored in a **node** that contains:
    - The value of the item (data)
    - A pointer to the next node

- A list consists of two pieces of information
    - A pointer to the first node
    - The number of elements in the list

##### Ex:
Let data = [5, 10, 2, 3, 2]

It will be stored as:

{ 1000 next 3000
<br>
1020 data 5 }
<br>
{ 2000 next NULL
<br>
2020 data 2 }
<br>
{ 3000 next 10
<br>
3020 data 2000 }
<br>
5->10->2

Linked list starts at 1000, ends at the node for next is NULL

### Linked Lists `insert`

- Suppose we want to insert value 34 at index 2 in the LL:
<img src="assets/LL1.png">

- Need to create a new node to store the new value
<img src="assets/LL2.png">

- Then, set next pointer of the new node to the `next` pointer of the node at index 1:
<img src="assets/LL3.png">

- Then, set `next` pointer of the node currently at index 1 to point to the new node
<img src="assets/LL4.png">

In [ ]:
%%file LL.h

#if !defined(LL_H)

typedef struct node {
    struct node *next; //Data structure with pointer to itself
    int data;
} node;

typedef struct LL {
    node *head;
    int sz;
} LL;

/*
- LL will contain the head pointer: the address of the 1st node in the LL
- Each node has data, as well as the address of the next node

Picture:
   123->567->123->NULL
    ^
    |
LL[head, sz]

- The 567 node is the data 567, and the address of the 123 node
*/

void create_LL_from_data(LL **p_LL, int *data_arr, int size);
void create_node(node** n, int data);

#endif

In [ ]:
%%file LL.c

#include "LL.h"
#include <stdlib.h>
#include <stdio.h>

// Creating a node
void create_node(node** n, int data) {
    *p_n = (node*)malloc(sizeof(node));
    (*p_n)->data = data;
    (*p_n)->next = NULL;
}

// Creating a linked list from data
void create_LL_from_data(LL **p_LL, int *data_arr, int size) {
    *p_LL = (LL*)malloc(sizeof(LL));
    (*p_LL)->head = NULL;
    (*p_LL)->sz = 0;

    int i = 0; //Nodes inserted so far
    node *prev = NULL;
    while (i < size) {
        //Current node we are processing
        node* cur;
        create_node(&cur, data_arr[0]);

        if (i == 0) {
            (*p_LL)->head = cur; //For the head/first index
        } else {
            prev->next = cur; //For the next node indices
        }
        prev = cur; //Make the previous node to the new current after inserting
        i++;
    }
}

void get_index(LL *p_LL, int i) {
    if (p_LL->sz >= i) {
        printf("ERROR: canot get element at index %d\n", i);
        exit(1);
    }
    int j = 0;
    node* cur = p_LL->head;

    while (j < i) {
        cur = cur->next;
    }
    //After running the while loop, cur is the node that contains the i'th element
    return cur->data;
}

void append(LL *p_LL, int data) {
    //Get to the last element, replace NULL with new element

    //If no head (1st elem), make a new node at the head
    if (p_LL->head == NULL) {
        node* new_node = (node*)malloc(sizeof(node));
        new_node->data = data;
        //or create_node(&new_node, data);
        p_LL->head = new_node;
        return;
    }

    //Advancing the current node to the last node
    node *cur = p_LL->head;
    while (cur->next != NULL) {
        cur = cur->next;
    }
    // Now cur->next is NULL, so need to replace cur->next with new data that contains data
    node* new_node = (node*)malloc(sizeof(node));
    new_node->data = data;
    cur->next = new_node;
    p_LL->sz += 1;
}

// Deleting a singular node
void destroy_node(node *node){
    free(node);
}

// Freeing the entire linked list
void destroy_list(LL *list){
    node *current = list->head;
    node *next;
    while(current != NULL){
        next = current->next;
        destroy_node(current);
        current = next;
    }
    free(list);
}

void print_list(LL *list){
    node *current = list->head;
    while(current != NULL){
        printf("%d", current->data);
        current = current->next;
        if(current != NULL){
            printf(" -> ");
        }
    }
    printf("\n");
}

void delete_list(LL *list, int data){
    node *current = list->head;
    node *prev = NULL;
    while(current != NULL){
        if(current->data == data){
            if(prev == NULL){
                list->head = current->next;
            }else{
                prev->next = current->next;
            }
            destroy_node(current);
            return;
        }
        prev = current;
        current = current->next;
    }
}

void delete_list_ind(LL *list, int index){
    node *current = list->head;
    node *prev = NULL;
    int i = 0;
    while(current != NULL){
        if(i == index){
            if(prev == NULL){
                list->head = current->next;
            }else{
                prev->next = current->next;
            }
            destroy_node(current);
            return;
        }
        prev = current;
        current = current->next;
        i++;
    }
}

//(call using create_LL_from_data(&p_LL, data_arr, size))
int main() {
    LL *p_LL;
    int data_arr[] = {1, 2, 3, 4}
    int size = sizeof(data_arr)/sizeof(data_arr[0]);
    
    create_LL_from_data(&p_LL, data_arr, size);

    return 0;
}